### Code Indexer

This takes in the dir url, and return a dictionary where key=filepath, value=encoded file

In [ ]:
import requests
import base64
import urllib
from urllib.parse import urlparse
import os
from dotenv import load_dotenv

load_dotenv()

MY_AUTH = (os.getenv("GITHUB_USER"), os.getenv("GITHUB_PAT"))

def parse_url(repo_url):
    parts = urlparse(repo_url).path.split("/")
    owner = parts[1]
    repo = parts[2]
    return owner, repo

def get_tree(owner, repo, branch='master'):
    response = requests.get(f'https://api.github.com/repos/{owner}/{repo}/git/trees/{branch}?recursive=1', auth=MY_AUTH)
    if response.status_code == 200:
        repo_tree = response.json()
        return [item for item in repo_tree['tree'] if item['path'].startswith('app/') and item['type'] == 'blob']
    else:
        return []

def get_content(owner, repo, file_sha):
    response = requests.get(f'https://api.github.com/repos/{owner}/{repo}/git/blobs/{file_sha}', auth=MY_AUTH)
    if response.status_code == 200:
        file_content = response.json()
        # base64.b64decode(file_content["content"]).decode('utf-8')
        return file_content["content"]
    else:
        return ''

def get_app_files(owner, repo, branch='master'):
    app_files = {}
    app_tree = get_tree(owner, repo, branch)
    for item in app_tree:
        file_content = get_content(owner, repo, item['sha'])
        app_files[item['path']] = file_content
    return app_files

dir_url = "https://github.com/alexwohlbruck/cat-facts/tree/master/app"
owner,repo = parse_url(dir_url)
tree = get_tree(owner, repo)
app_files = get_app_files(owner, repo)

print(app_files.keys())


In [43]:
import codecs
import base64
# codecs.decode(encoded_file, 'base64').decode()

base64.b64decode(encoded_file).decode('utf-8')

'var mongoose = require(\'mongoose\');\nvar Schema = mongoose.Schema;\n\nvar MessageSchema = new Schema({\n    text: {type: String, required: true},\n    number: {type: String, required: true},\n    type: {type: String, enum: [\'incoming\', \'outgoing\']}\n}, {\n    timestamps: true\n});\n\nMessageSchema.index({ "createdAt": 1 }, { expireAfterSeconds: (60 * 60 * 24 * 7 * 2) });\n\nvar Message = mongoose.model(\'Message\', MessageSchema);\n\nmodule.exports = Message;'